What is the FAIR Plan?
The California FAIR Plan (Fair Access to Insurance Requirements) is a state-mandated insurance pool created in 1968 to provide basic fire insurance coverage to homeowners who cannot get insured through the private market.

It is not a government-funded program — it’s funded by a consortium of private insurance companies — but it operates under state oversight and exists purely as a backstop for the market.

🧯 Why is it Called "Last Resort"?
1. Limited Coverage
The FAIR Plan only covers fire-related damages (and in some cases, smoke or vandalism), unlike standard homeowners insurance, which includes liability, theft, water damage, etc.
Homeowners often have to purchase a second policy ("wrap-around coverage") to get full protection.

2. Higher Cost, Fewer Benefits
It’s often more expensive and less comprehensive than traditional insurance.
Deductibles may be higher, and coverage limits lower.

3. Eligibility Only When Denied
You typically have to be denied by multiple private insurers to qualify.
This makes it a plan of last resort — homeowners don't choose it unless they have no other option.

In [ ]:
import pandas as pd

df = pd.read_csv('aggregated_fires_insurance_dataset.csv')
df = df.drop(columns=['Unnamed: 0'])
df = df[df['Year'] > 2015]
df

,County,Year,Voluntary Market: Number of New Policies,Voluntary Market: Number of Renewed Policies,Voluntary Market: Number of Non-Renewed Policies,FAIR Plan: Number of New Policies,FAIR Plan: Number of Renewed Policies,Difference-in-Conditions: Number of New Policies,Difference-in-Conditions: Number of Renewed Policies,Total Acres Burned
0,Alameda,2023,"25,158","318,139","26,698",1021,"2,359",878,"1,502",80.0
1,Alameda,2022,"32,872","317,670","32,294",788,"2,137",693,"1,129",802.0
2,Alameda,2021,"40,972","315,157","39,636",722,"1,899",626,783,0.0
3,Alameda,2020,"37,118","318,576","32,905",727,"1,672",537,470,79685.8
4,Alameda,2019,"38,023","316,762","35,206",592,"1,660",0,0,533.0
...,...,...,...,...,...,...,...,...,...,...
525,Yuba,2020,"3,147","17,196","2,610",487,349,362,220,2467.0
526,Yuba,2019,"3,421","16,567","2,875",305,83,0,0,268.0
527,Yuba,2018,"3,027","16,427","2,675",37,47,0,0,0.0
528,Yuba,2017,"2,770","16,137","2,677",29,31,0,0,10981.0


Feature Engineering & Cleaning up the df

In [ ]:
insurance_cols = [
    "Voluntary Market: Number of Renewed Policies",
    "Voluntary Market: Number of New Policies",
    "Voluntary Market: Number of Non-Renewed Policies",
    "FAIR Plan: Number of Renewed Policies",
    "FAIR Plan: Number of New Policies",
    "Difference-in-Conditions: Number of Renewed Policies",
    "Difference-in-Conditions: Number of New Policies"
]

# Replace commas and dashes, then coerce errors into NaN
for col in insurance_cols:
    df[col] = (
        df[col]
        .replace({'-': '0', ',': ''}, regex=True)
        .apply(pd.to_numeric, errors='coerce')  # forces non-numeric to NaN
    )
insurance_cols.append('Total FAIR')
insurance_cols.append('Total DiC')
df['Total FAIR'] = df['FAIR Plan: Number of New Policies'] + df['FAIR Plan: Number of Renewed Policies']
df['Total DIC'] = df['Difference-in-Conditions: Number of New Policies'] + df['Difference-in-Conditions: Number of Renewed Policies']

df[insurance_cols] = df[insurance_cols].fillna(0)

In [ ]:
pop_df = pd.read_csv('California Counties by Population 2025.csv')
pop_df['county'] = pop_df['county'].str.replace(' County', '')
pop_df = pop_df.rename(columns={'county': 'County'})
pop_df = pop_df[['County', 'pop2020']]
pop_df

,County,pop2020
0,Los Angeles,9992813
1,San Diego,3295298
2,Orange,3185734
3,Riverside,2422993
4,San Bernardino,2183391
5,Santa Clara,1931168
6,Alameda,1680466
7,Sacramento,1586566
8,Contra Costa,1166069
9,Fresno,1009552


In [ ]:
df = df.merge(pop_df, on='County')
df

,County,Year,Voluntary Market: Number of New Policies,Voluntary Market: Number of Renewed Policies,Voluntary Market: Number of Non-Renewed Policies,FAIR Plan: Number of New Policies,FAIR Plan: Number of Renewed Policies,Difference-in-Conditions: Number of New Policies,Difference-in-Conditions: Number of Renewed Policies,Total Acres Burned,Total FAIR,Total DiC,pop2020
0,Alameda,2023,25158,318139,26698,1021,2359,878,1502,80.0,3380,2380,1680466
1,Alameda,2022,32872,317670,32294,788,2137,693,1129,802.0,2925,1822,1680466
2,Alameda,2021,40972,315157,39636,722,1899,626,783,0.0,2621,1409,1680466
3,Alameda,2020,37118,318576,32905,727,1672,537,470,79685.8,2399,1007,1680466
4,Alameda,2019,38023,316762,35206,592,1660,0,0,533.0,2252,0,1680466
...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,Yuba,2020,3147,17196,2610,487,349,362,220,2467.0,836,582,81958
460,Yuba,2019,3421,16567,2875,305,83,0,0,268.0,388,0,81958
461,Yuba,2018,3027,16427,2675,37,47,0,0,0.0,84,0,81958
462,Yuba,2017,2770,16137,2677,29,31,0,0,10981.0,60,0,81958


##Research Question: How do increasing wildfire severities (measured in acres burned) influence the rise of last-resort and non-traditional insurance plans in California counties over time?

**Voluntary Market:** Number of Non-Renewed Policies
Counts homeowners whose insurance policies were not renewed by private insurers.
This reflects insurer retreat: when companies deem areas too risky and pull out.
Tends to lag behind big wildfire years (e.g., major fire in 2018 → large non-renewals in 2019–2020).
Interpreted as a signal of insurance market fragility under climate pressure.


**FAIR Plan:** Number of New Policies
Tracks new enrollments in California’s last-resort fire insurance program.
The FAIR Plan is state-backed and typically only used when private options disappear.
Its growth reflects desperation and systemic retreat by the private market.
Shows a steadily increasing trend regardless of year-to-year fire fluctuations — indicating long-term structural stress.

**Difference-in-Conditions (DIC):** Number of New Policies
Counts new purchases of supplemental fire-only insurance when a homeowner’s standard policy excludes wildfire coverage.
DIC policies are issued by private insurers but only cover what the main policy no longer does — usually fire.
Their growth signals a partial withdrawal by insurers: rather than exiting completely, they strip fire protection and push that risk onto a second policy.
DIC growth reflects a subtler form of market breakdown — one where coverage becomes fragmented, costly, and harder for homeowners to manage.

In [ ]:
import pandas as pd
import plotly.express as px

yearly = df.groupby("Year").agg({
    "Total Acres Burned": "sum",
    "Voluntary Market: Number of New Policies": "sum",
    "Voluntary Market: Number of Renewed Policies": "sum",
    "Voluntary Market: Number of Non-Renewed Policies": "sum",
    "FAIR Plan: Number of New Policies": "sum",
    "FAIR Plan: Number of Renewed Policies": "sum"
}).reset_index()

# Add 1-year lag to non-renewed policies
yearly["Lagged Non-Renewed Policies"] = yearly["Voluntary Market: Number of Non-Renewed Policies"].shift(-1)

# Melt long-form DataFrame with original + lagged column
df_long = yearly.melt(
    id_vars="Year",
    value_vars=["Total Acres Burned", "Lagged Non-Renewed Policies"],
    var_name="Metric",
    value_name="Count"
)
# Clean metric labels for readability
df_long["Metric"] = df_long["Metric"].replace({
    "Lagged Non-Renewed Policies": "Voluntary Market: Non-Renewed Policies (Lagged 1 Year)"
})

fig = px.line(
    df_long,
    x="Year",
    y="Count",
    color="Metric",
    markers=True,
    title="Wildfire Impact on Insurance Market (Statewide, with Lagged Non-Renewals)"
)
fig.show()


In [ ]:
yearly.rename(columns={'Lagged Non-Renewed Policies': 'Non-Renewal', 'Total Acres Burned': 'Wildfire'}, inplace=True)
yearly[['Non-Renewal', 'Wildfire']].corr()

,Non-Renewal,Wildfire
Non-Renewal,1.000000,0.678932
Wildfire,0.678932,1.000000


**🪓 A. Wildfire Severity → Lagged Voluntary Market Retreat:**

Visual trends and statistical analysis both point to a clear relationship between wildfire activity and insurer behavior — specifically, non-renewals in the voluntary market tend to increase in the year after large fires.
For instance, the 2018 Camp Fire season correlates strongly with non-renewal spikes in 2019–2020.
This pattern reflects the real-world timing of the insurance cycle:
Policies are typically renewed once per year.
Insurers assess wildfire risk after the season ends, then decide whether to pull out.
This reactive pattern is confirmed statistically:
A correlation coefficient of 𝑟 = 0.679 shows a strong positive linear relationship between acres burned in a given year and the number of non-renewed policies the following year.
That’s a meaningful signal in real-world data, where many factors are at play.
It confirms that wildfires are a leading indicator of insurance market withdrawal.
This tells us the voluntary market is not proactive in managing wildfire risk — it waits for damage, then retreats.



**🧯 B. FAIR Plan Growth Is Persistent, Not Just Reactive:**

In contrast to the volatile behavior of the voluntary market, FAIR Plan enrollments steadily increase over time, regardless of individual fire seasons.
This suggests:
Many homeowners are stuck in the FAIR Plan after being dropped — and not returning to the private market.
Others may be turning to it proactively, knowing they live in high-risk zones where private coverage is unstable or unavailable.
Rather than reacting to each fire season, the FAIR Plan seems to be evolving into a long-term structural safety net. Its steady rise reflects a systemic shift in how fire risk is insured — with the state quietly absorbing more and more responsibility as the private market pulls away.



##Voluntary Market: % Renewal vs. Major Wildfires

In [ ]:

yearly = df.groupby("Year").agg({
    "Voluntary Market: Number of Renewed Policies": "sum",
    "FAIR Plan: Number of New Policies": "sum",
    "FAIR Plan: Number of Renewed Policies": "sum",
    "Difference-in-Conditions: Number of New Policies": "sum",
    "Difference-in-Conditions: Number of Renewed Policies": "sum"
}).reset_index()

yearly["Total FAIR"] = yearly["FAIR Plan: Number of New Policies"] + yearly["FAIR Plan: Number of Renewed Policies"]
yearly["Total DIC"] = yearly["Difference-in-Conditions: Number of New Policies"] + yearly["Difference-in-Conditions: Number of Renewed Policies"]

# Compute percent renewed
yearly["% Renewed"] = 100 * yearly["Voluntary Market: Number of Renewed Policies"] / (
    yearly["Voluntary Market: Number of Renewed Policies"] +
    yearly["Total FAIR"] +
    yearly["Total DIC"]
)

df_pct_renewed = yearly[["Year", "% Renewed"]]

# Major wildfires
wildfires = {
    2017: ("Tubbs Fire", 36807),
    2018: ("Camp Fire", 153336),
    2020: ("August Complex", 1032648),
    2021: ("Dixie Fire", 963309)
}

fig = px.line(
    df_pct_renewed,
    x="Year",
    y="% Renewed",
    markers=True,
    title="Percent of Voluntary Renewals Among All Fire Policies (Annotated with Major Wildfires)"
)

# Add wildfire annotations
for year, (name, acres) in wildfires.items():
    y_val = df_pct_renewed.loc[df_pct_renewed["Year"] == year, "% Renewed"].values[0]
    label = f"{name} ({acres:,} acres)"
    fig.add_annotation(
        x=year,
        y=y_val,
        text=label,
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=-40
    )

fig.update_layout(yaxis_title="% Renewed", xaxis=dict(dtick=1))
fig.show()


**Analysis: Voluntary Market Shrinks as Wildfire Risk Grows**

This graph tracks the percentage of all fire insurance policies that are voluntary renewals, contextualized with major wildfires and their size. What emerges is a clear downward trend in the share of policies renewed through the traditional private market — a signal of deepening strain.

🔥 Wildfires Trigger Structural Decline
Each major wildfire is followed by a noticeable drop in the percentage of voluntary renewals:
After the 2017 Tubbs Fire (36,807 acres) and the 2018 Camp Fire (153,336 acres), the decline begins.
The August Complex Fire in 2020 and the Dixie Fire in 2021, which burned over a million and nearly a million acres respectively, coincide with sharp drops in voluntary market share.

This pattern suggests that insurers are steadily pulling back, choosing not to renew policies in high-risk areas after catastrophic fire seasons. Unlike raw renewal counts, this percentage metric adjusts for growth in alternative options like FAIR and DIC, giving a clearer sense of relative market retreat.

## FAIR Plan in the most fire proned counties

In [ ]:
df_total = df.groupby("County")['Total Acres Burned'].sum().reset_index()
pop_by_county = df.groupby("County")['pop2020'].mean().reset_index()


df_total = df_total.merge(pop_by_county, on="County")
df_total = df_total[df_total["pop2020"] > 300_000]


df_total["Fire Density"] = df_total["Total Acres Burned"] / df_total["pop2020"]
df_total = df_total.sort_values(by="Fire Density", ascending=False)

df_total

,County,Total Acres Burned,pop2020,Fire Density
41,Santa Barbara,243744.5,448424.0,0.543558
48,Sonoma,249669.0,488282.0,0.511321
26,Monterey,209244.0,438322.0,0.477375
9,Fresno,263261.5,1009552.0,0.260771
55,Ventura,217128.0,843371.0,0.257453
47,Solano,110090.0,452723.0,0.243173
53,Tulare,99581.0,473914.0,0.210125
49,Stanislaus,91194.8,553257.0,0.164833
30,Placer,66708.0,405927.0,0.164335
14,Kern,135065.0,905910.0,0.149093


In [ ]:
most_impacted_counties = df_total['County'].head(4).tolist()

df_filtered = df[df["County"].isin(most_impacted_counties)]

# Convert to long form for plotting
df_long_county = df_filtered.melt(
    id_vars=["Year", "County"],
    value_vars=["Total FAIR", "Total DiC"],
    var_name="Metric",
    value_name="Count"
)

fig = px.line(
    df_long_county,
    x="Year",
    y="Count",
    color="Metric",
    facet_col="County",
    facet_col_wrap=1,
    markers=True,
    title="County-Level Trends: FAIR Plan Enrollment vs Wildfire Severity"
)
fig.update_layout(legend_title_text="Metric", height=900)
fig.update_layout(
    legend=dict(
        x=0.02,
        y=0.98,
        bgcolor='rgba(255,255,255,0.6)',
        bordercolor='black',
        borderwidth=1
    )
)
fig.show()

**📊 What the Graph Shows:**
This graph tracks the total FAIR Plan and DiC enrollment (renewed + new policies) over time in five counties with significant populations (>300,000) and high fire density relative to population:
Fresno
Monterey
Santa Barbara
Sonoma
Ventura
Each subplot highlights year-by-year FAIR Plan and DiC trends in one county, revealing how each area responds to increasing climate risk and insurance market pressure.

🔥 Key Insights:
1. Dual Rise in Emergency Coverage
All counties show clear upward trends in both FAIR and DIC policies since 2017–2018, reflecting sustained pressure on the voluntary insurance market.
This dual increase suggests residents are not just getting dropped — they are being pushed into fragmented alternatives, where one plan covers fire (DIC) and another may cover the rest.

2. Different Roles, Same Cause
FAIR Plan growth often occurs in counties with more complete voluntary insurance collapse — it replaces full policies when no insurer will step in.
DIC policies grow where insurers still offer some coverage, but strip out wildfire protection — forcing homeowners to patch the gap.

3. Policy and Real-World Implications
This graph shows that California's insurance market is becoming less unified, more fragile, and more expensive to navigate:
What used to be covered by one insurer now takes two or more policies — each with its own limitations.
The rise in FAIR + DIC is a warning: risk isn't just growing — it's being offloaded onto individuals and the state.

In [ ]:
import pandas as pd
import plotly.graph_objects as go

df["Total Policies"] = (
    df["Voluntary Market: Number of Renewed Policies"] +
    df["Voluntary Market: Number of New Policies"] +
    df["Total FAIR"] +
    df["Difference-in-Conditions: Number of Renewed Policies"] +
    df["Difference-in-Conditions: Number of New Policies"]
)
df["FAIR Share (%)"] = 100 * df["Total FAIR"] / df["Total Policies"]

# Top 10 counties by fire/pop density
county_density = df.groupby("County").agg({
    "Total Acres Burned": "sum",
    "pop2020": "mean"
}).reset_index()
county_density["Fire/Pop Ratio"] = county_density["Total Acres Burned"] / county_density["pop2020"]
top_fire_dense_counties = county_density.sort_values(by="Fire/Pop Ratio", ascending=False).head(10)["County"].tolist()

# Heatmap data
filtered_df = df[df["County"].isin(top_fire_dense_counties)]
heat_df = filtered_df.groupby(["County", "Year"])["FAIR Share (%)"].mean().reset_index()
pivot_df = heat_df.pivot(index="County", columns="Year", values="FAIR Share (%)")

# Plot
fig = go.Figure(
    data=go.Heatmap(
        z=pivot_df.values,
        x=pivot_df.columns,
        y=pivot_df.index,
        colorscale="Reds",
        colorbar=dict(title="% FAIR Plan Share")
    )
)

fig.update_layout(
    title="Rising FAIR Plan Reliance in Most Fire-Dense Counties (Top 10 by Fire-to-Pop Ratio)",
    xaxis_title="Year",
    yaxis_title="County",
    height=600
)

fig.show()


**🔥 Heatmap Analysis:**
Rising Dependence on FAIR Plan in California's Largest Counties (2015–2023)
This heatmap visualizes the percentage of all homeowner insurance policies in each county that are provided by the FAIR Plan, California’s last-resort fire insurance program, for counties with populations over 300,000.
Each cell represents the FAIR Plan's share of total active policies in a county during a given year — including both new and renewed FAIR Plan enrollments.

**Key Trends:**
1. Steady, Widespread Growth
Virtually every county shows a consistent upward trend in FAIR Plan share over time.
This suggests that the FAIR Plan is not just a short-term emergency program — it’s becoming a mainstay of the market.

2. Hot Zones of FAIR Dependence
Placer, San Bernardino, and Sacramento Counties stand out with FAIR Plan shares rising above 6–8% by 2023.
These counties are known wildfire zones (e.g. near Tahoe National Forest or inland mountain foothills), showing how geographic risk drives structural insurance displacement.